In [83]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from math import sqrt

from xgboost import XGBRegressor, plot_importance
import lightgbm as lgb
from lightgbm import LGBMRegressor

### LOAD TRAIN AND TEST DATA

In [84]:
# Read training data
data = pd.read_csv("Best_buy_Data_preprocessed_final.csv")
print(data.shape)
data.head()

(879429, 16)


,encoded_sku_id,sales_date,subclass_name,class_name,ml_name,category_name,retail_price,promo_price,competitor_price,inventory,daily_units,promo_price_avbl,competitor_price_avbl,month,week,day
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,21,Sunday
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Monday
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Tuesday
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Wednesday
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,6,22,Thursday


In [85]:
# Read test data
valid_data = pd.read_csv("Best_buy_Validation_Data_preprocessed.csv")
valid_data.drop(columns = 'forecasted units', inplace = True)
print(valid_data.shape)
valid_data.head()

(3773, 16)


,encoded_sku_id,sales_date,subclass_name,class_name,ml_name,category_name,retail_price,promo_price,competitor_price,inventory,daily_units,promo_price_avbl,competitor_price_avbl,month,week,day
0,1,2022-08-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0,8,31,Monday
1,1,2022-08-02,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,1.0,0,0,8,31,Tuesday
2,1,2022-08-03,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,69.95,Fully-Stocked,0.0,0,1,8,31,Wednesday
3,1,2022-08-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0,0,8,31,Thursday
4,1,2022-08-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,1.0,0,0,8,31,Friday


In [86]:
# Get the start and end for test data
valid_start_date = valid_data['sales_date'].min()
valid_end_date = valid_data['sales_date'].max()

In [87]:
# Concatenate data to create lag/shift features
data = pd.concat([data,valid_data], ignore_index=True)
data = data.sort_values(by = ['encoded_sku_id','sales_date']).reset_index(drop=True)
print(data.shape)
data.head()

(883202, 16)


,encoded_sku_id,sales_date,subclass_name,class_name,ml_name,category_name,retail_price,promo_price,competitor_price,inventory,daily_units,promo_price_avbl,competitor_price_avbl,month,week,day
0,1,2017-05-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,21,Sunday
1,1,2017-05-29,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Monday
2,1,2017-05-30,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Tuesday
3,1,2017-05-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,5,22,Wednesday
4,1,2017-06-01,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,0.0,0.0,0.0,6,22,Thursday


### MODELING

In [88]:
# Create 7 days lag features
for i in range(1,8):
  data['daily_units_lag_'+ str(i) + 'd'] = data.groupby('encoded_sku_id')['daily_units'].shift(i)

data[data['encoded_sku_id'] == 23][['encoded_sku_id','sales_date','daily_units','daily_units_lag_1d','daily_units_lag_7d']].tail(5)

,encoded_sku_id,sales_date,daily_units,daily_units_lag_1d,daily_units_lag_7d
39027,23,2022-08-03,2.0,3.0,0.0
39028,23,2022-08-04,0.0,2.0,2.0
39029,23,2022-08-05,0.0,0.0,4.0
39030,23,2022-08-06,3.0,0.0,0.0
39031,23,2022-08-07,0.0,3.0,1.0


In [89]:
# Drop the first and last rows with NA values
data = data.dropna().reset_index(drop=True)
data.head()

,encoded_sku_id,sales_date,subclass_name,class_name,ml_name,category_name,retail_price,promo_price,competitor_price,inventory,...,month,week,day,daily_units_lag_1d,daily_units_lag_2d,daily_units_lag_3d,daily_units_lag_4d,daily_units_lag_5d,daily_units_lag_6d,daily_units_lag_7d
0,1,2017-06-04,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,...,6,22,Sunday,7.0,4.0,0.0,0.0,0.0,0.0,0.0
1,1,2017-06-05,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,...,6,23,Monday,4.0,7.0,4.0,0.0,0.0,0.0,0.0
2,1,2017-06-06,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,...,6,23,Tuesday,2.0,4.0,7.0,4.0,0.0,0.0,0.0
3,1,2017-06-07,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,...,6,23,Wednesday,4.0,2.0,4.0,7.0,4.0,0.0,0.0
4,1,2017-06-08,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,89.99,89.99,Fully-Stocked,...,6,23,Thursday,3.0,4.0,2.0,4.0,7.0,4.0,0.0


#### CREATE TRAIN & TEST DATA

In [90]:
# One Hot Encode features
categorical_feats = ['encoded_sku_id','subclass_name','class_name','ml_name','category_name','inventory','month','week','day']
final_data = pd.get_dummies(data, columns=categorical_feats)

# Convert sales to datetime
final_data['sales_date'] = pd.to_datetime(final_data['sales_date'])

# Train & test split
train = final_data[final_data['sales_date'] < valid_start_date]
test = final_data[(final_data['sales_date'] >= valid_start_date)]

train = train.drop(columns = 'sales_date')
test = test.drop(columns = 'sales_date')

In [91]:
print(train.shape)
print(test.shape)

(875404, 817)
(3773, 817)


In [92]:
# Create data for model input
X_train = train.drop(columns = 'daily_units')
y_train = train['daily_units']

X_test = test.drop(columns = 'daily_units')
y_test = test['daily_units']

X_train.head()

,retail_price,promo_price,competitor_price,promo_price_avbl,competitor_price_avbl,daily_units_lag_1d,daily_units_lag_2d,daily_units_lag_3d,daily_units_lag_4d,daily_units_lag_5d,...,week_51,week_52,week_53,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,89.99,89.99,89.99,0.0,0.0,7.0,4.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,89.99,89.99,89.99,0.0,0.0,4.0,7.0,4.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,89.99,89.99,89.99,0.0,0.0,2.0,4.0,7.0,4.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,89.99,89.99,89.99,0.0,0.0,4.0,2.0,4.0,7.0,4.0,...,0,0,0,0,0,0,0,0,0,1
4,89.99,89.99,89.99,0.0,0.0,3.0,4.0,2.0,4.0,7.0,...,0,0,0,0,0,0,0,1,0,0


#### FITTING BASE MODEL

In [93]:
lgb_model = LGBMRegressor(seed=1)
lgb_model.fit(X_train,y_train)

LGBMRegressor(seed=1)

#### FITTING BEST MODEL - CROSS VALIDATION

In [94]:
# # creating a dict of grids
# params = dict()
# # values for iteration
# params['n_estimators'] = [10, 50, 100, 500]
# # values for learning rate
# params['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]
# # values for the sample
# params['subsample'] = [0.5, 0.7, 1.0]
# # values for teh depth of tree
# params['max_depth'] = [3, 4, 5]

In [95]:
# Create parameters for cross validation 
params ={'n_estimators' : np.arange(100,1100,100),
        'learning_rate' : np.arange(0.005,0.1,0.005),
        'max_depth': np.arange(1,10), 
        #  'min_child_samples': sp_randint(100, 500), 
        #  'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
        'subsample': np.arange(0.01, 1.0, 0.1),
        'sub_feature': np.arange(0.3, 1, 0.1),
        # 'bagging_freq': np.arange(0,5,1),
        'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
        'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [96]:
# Hyperparameter Tuning
lgb_model = LGBMRegressor(seed=1)

# # applying the gridsearchcv method
# grid_search = GridSearchCV(estimator=lgb_model, param_grid=grid, n_jobs=-1, cv=cv, scoring='neg_root_mean_squared_error')

best_model = RandomizedSearchCV(estimator= lgb_model,
                        param_distributions = params,
                        scoring='neg_root_mean_squared_error',
                        n_iter = 5, cv = 3, n_jobs=-1,
                        verbose = 1, random_state=0, error_score='raise')

# storing the values
model_result = best_model.fit(X_train, y_train)
# printing the best parameters
print("Cross Validated RMSE: %f using %s" % (model_result.best_score_, model_result.best_params_))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


MemoryError: Unable to allocate 447. MiB for an array with shape (804, 583602) and data type uint8

In [ ]:
# Model Prediction and Evaluation
y_pred = lgb_model.predict(X_test)
y_pred = model_result.predict(X_test)

rmse = sqrt(mean_squared_error(y_test, y_pred))

print("Model Test RMSE is:", round(rmse,2))

Model Test RMSE is: 5.0


In [ ]:
y_pred[:50]

In [ ]:
y_test.head(50)